In [1]:
pip install haystack-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.9/351.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [2]:
!pip install "sentence-transformers>=3.0.0" "huggingface_hub>=0.23.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 864.6 kB/s eta 0:00:00


In [3]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.utils import Secret
from haystack.components.generators import HuggingFaceAPIGenerator

In [4]:
from haystack.components.writers import DocumentWriter
from haystack.components.converters import MarkdownToDocument, PyPDFToDocument, TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from pathlib import Path

In [5]:
#Preprocessing-Pipeline bauen
document_store = InMemoryDocumentStore()
file_type_router = FileTypeRouter(mime_types=["text/plain", "application/pdf", "text/markdown"])
text_file_converter = TextFileToDocument()
document_joiner = DocumentJoiner()

In [6]:
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_by="word", split_length=150, split_overlap=50)

In [7]:
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)

In [8]:
preprocessing_pipeline = Pipeline()
preprocessing_pipeline.add_component(instance=file_type_router, name="file_type_router")
preprocessing_pipeline.add_component(instance=text_file_converter, name="text_file_converter")
preprocessing_pipeline.add_component(instance=document_joiner, name="document_joiner")
preprocessing_pipeline.add_component(instance=document_cleaner, name="document_cleaner")
preprocessing_pipeline.add_component(instance=document_splitter, name="document_splitter")
preprocessing_pipeline.add_component(instance=document_embedder, name="document_embedder")
preprocessing_pipeline.add_component(instance=document_writer, name="document_writer")

In [9]:
preprocessing_pipeline.connect("file_type_router.text/plain", "text_file_converter.sources")
preprocessing_pipeline.connect("text_file_converter", "document_joiner")
preprocessing_pipeline.connect("document_joiner", "document_cleaner")
preprocessing_pipeline.connect("document_cleaner", "document_splitter")
preprocessing_pipeline.connect("document_splitter", "document_embedder")
preprocessing_pipeline.connect("document_embedder", "document_writer")

🚅 Components
  - file_type_router: FileTypeRouter
  - text_file_converter: TextFileToDocument
  - document_joiner: DocumentJoiner
  - document_cleaner: DocumentCleaner
  - document_splitter: DocumentSplitter
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.text/plain -> text_file_converter.sources (List[Path])
  - text_file_converter.documents -> document_joiner.documents (List[Document])
  - document_joiner.documents -> document_cleaner.documents (List[Document])
  - document_cleaner.documents -> document_splitter.documents (List[Document])
  - document_splitter.documents -> document_embedder.documents (List[Document])
  - document_embedder.documents -> document_writer.documents (List[Document])

In [ ]:
#from sentence_transformers import SentenceTransformer

In [ ]:
#pip uninstall sentence-transformers

Found existing installation: sentence-transformers 3.0.1
Uninstalling sentence-transformers-3.0.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/sentence_transformers-3.0.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/sentence_transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled sentence-transformers-3.0.1


In [ ]:
#pip install "sentence-transformers>=3.0.0"

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)


In [10]:
output_dir = "/content/"  # Pfad zum Verzeichnis, wo sich die Dateien befinden
data_files = list(Path(output_dir).glob("*.txt"))  # alle txt.-Dateien

for data_file in data_files:
    preprocessing_pipeline.run({"file_type_router": {"sources": [str(data_file)]}})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/269 [00:00<?, ?it/s]

Batches:   0%|          | 0/1157 [00:00<?, ?it/s]

Batches:   0%|          | 0/4668 [00:00<?, ?it/s]

In [11]:
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

In [12]:
retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_template)

In [13]:
#token bei Hugging-Face unter Access-Token erstellen, wichtig: write!
generator = HuggingFaceAPIGenerator(api_type="serverless_inference_api",
                                    api_params={"model": "mistralai/Mixtral-8x7B-Instruct-v0.1"},
                                    token=Secret.from_token("<add_your_api_token_here>"))

In [14]:
rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", generator)
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceAPIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [15]:
#Question 1
question = "What are all the marriages and child relationships following the final chapter? In the last chapter of Naruto, some characters got married, and some of them had kids. What is the entire husband-wife-kids set?"

In [16]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [17]:
print(results["llm"]["replies"])

[" The last chapter of Naruto, chapter 700, shows that Naruto and Hinata got married and had two children, Boruto and Himawari. Sasuke and Sakura also got married and had a daughter, Sarada. Shikamaru and Temari got married and had a daughter, Shikadai. Kiba and Tamaki got married and had a son, Akamaru. Rock Lee and Tenten did not get married, but they are shown to be close friends. Choji and Karui got married and had a daughter, Chocho. Neji and Tenten did not get married, as Neji died in the war. Shino and Ino did not get married, as Shino was shown to be single. Kankuro and Temari did not get married, as Temari married Shikamaru. Gaara did not get married, as he was shown to be single. Hinata's sister, Hanabi, did not get married, as she was shown to be single. Sai and Ino did not get married, as Sai was shown to be single. Sasuke's brother, Itachi, did not get married, as he died before the series ended. Kakashi did not get married, as he was shown to be single. Guy did not get ma

In [18]:
#Question 2
question = "Why does Naruto use his mother's family name? Is there any logical and canonical reason to why Naruto used his mother's family name, Uzumaki, instead of his father's family name?"

In [19]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [20]:
print(results["llm"]["replies"])

[" Yes, there is a canonical reason to why Naruto used his mother's family name, Uzumaki, instead of his father's family name. Naruto's father, Minato Namikaze, was the Fourth Hokage of Konohagakure, and Naruto inherited his father's blond hair and blue eyes. However, Naruto's mother, Kushina Uzumaki, was a relative to the Uzumaki clan from Uzushiogakure in the Land of Whirlpools, who used the spiral pattern as both their clan and village symbol. Naruto's full name is also a reference to the Naruto whirlpools (鳴門の渦潮, Naruto no uzushio), named after the city in the Tokushima Prefecture in Japan. Naruto's mother, Kushina Uzumaki, is a relative to the Uzumaki clan from Uzushiogakure in the Land of Whirlpools, who used the spiral pattern as both their clan and village symbol. According to the databook(s), Naruto's family name is Uzumaki, not Namikaze. Therefore, Naruto uses his mother's family name, Uzumaki, instead of his father's family name."]


In [21]:
#Question 3
question = "What will happen to the tailed beast if the jinchuruki is killed? It is shown in the anime that the jinchuruki is caught and the tailed beast is extracted and sealed into the Demonic statue by the Akatsuki. What will happen if the jinchuruki is killed somewhere else? In search for Tsunade arc, during the fight between Jiraiya-Tsunade and Orochimaru, Orochimaru tries to kill Naruto as he is the jinchuruki of the nine tails. What would have happened to the nine tails if he had killed Naruto?"

In [22]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [23]:
print(results["llm"]["replies"])

[" If the jinchuruki is killed, the tailed beast will not be extracted and sealed into the Demonic statue by the Akatsuki. Instead, the tailed beast will roam freely. In the scenario you mentioned, if Orochimaru had killed Naruto, the Nine-Tails would have been released and would have wreaked havoc. However, in the actual story, Tsunade sacrificed herself to protect Naruto from Orochimaru's attack."]


In [25]:
#Question 4
question = "When did Kakashi and Obito awaken their Mangekyou Sharingan? In chapter 604 of the manga, it is shown that Kakashi kills Rin. This fulfills the requirement to activate the Mangekyou Sharingan. So that should have been the moment when Kakashi was able to use the Mangekyou Sharingan. Also, the Yellow Flash was still alive when this happens, so this means that Naruto was not yet born. This clearly points to the fact that from the very beginning of the Naruto series, Kakashi had his Mangekyou Sharingan activated. When Kakashi first fought Itachi (when Itachi and Kisame appear at Konoha) and faced the Tsukuyomi he was devastated in a moment, even though he had his Mangekyou Sharingan active (though not used). But the next time they face (during the rescue mission of Gara), he was able to withstand it. Now my first question is, was Kakashi unaware of his Mangekyou Sharingan until he faced Itachi, or was he not proficient in using it. Also why in the Naruto series (I mean the series before the start of Shippuden) didn't Kakashi use his Mangekyou Sharingan? Now for my second question, in the same chapter, a few pages back, just when Kakashi strikes the blow at Rin, Obito's vision starts to fade as he can sense Rin dying. Does this mean Obito's Mangekyou Sharingan was awakened at the same time? We know that Kakashi and Obito shares the same other dimension because of the same Sharingan that the both use (not the same eye, but the eye of the same Uchiha). So does the same apply for their activation of Mangekyou Sharingan?"

In [26]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [27]:
print(results["llm"]["replies"])

[" Kakashi and Obito both awakened their Mangekyō Sharingan after the loss of a loved one. For Kakashi, this was when he was forced to kill his teammate Rin Nohara in Chapter 604 of the manga. For Obito, this was when he witnessed the death of Rin Nohara during the Third Shinobi World War. However, the exact moment when Obito's Mangekyō Sharingan activated is not explicitly shown in the given document. It is mentioned that Obito's vision started to fade as he sensed Rin dying, but this does not necessarily mean that his Mangekyō Sharingan activated at that moment. It is possible that it activated later, after Obito had fully processed Rin's death. As for Kakashi, it is mentioned that he was unaware of his Mangekyō Sharingan until he faced Itachi and was devastated by the Tsukuyomi. This suggests that he did not know how to use it proficiently at first, and may not have even been aware of its existence. The reason why Kakashi did not use his Mangekyō Sharingan in the Naruto series befor

In [28]:
#Question 5
question = "What happened to the Yin half of Kurama? Did Naruto get both halves of Kurama after fighting Sasuke, or did the other half of Kurama die when Minato was killed?"

In [29]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [30]:
print(results["llm"]["replies"])

[' The Yin half of Kurama was absorbed by Black Zetsu, becoming its jinchūriki. It did not die when Minato was killed, but it was later transferred into Naruto by Obito.']


In [31]:
#Question 6
question = "At what age did Kakashi Hatake become a Jounin? This answer on Yahoo! says he became a Jounin at the age of 13, which makes him the youngest Jounin. I have heard a lot of chatter on the subject, but I've never seen the answer stated outright (or with any proof). The youngest I've seen reference to was Kakashi. He was a chunin at 6, a jonin at 10, and joined ANBU Black Ops at 13. Itachi, on the other hand, became a chunin at 10 and became a Captain of ANBU Black Ops at 13. He's still pretty amazing, but Kakashi was younger. Is it true that he is the youngest Jounin? What is his real age when he became a Jounin? The magic number starts from at the age of 5"

In [32]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [33]:
print(results["llm"]["replies"])

[' According to the documents provided, Kakashi Hatake became a Jounin at the age of 13. This is consistent with the information that he was a chunin at 6, a jonin at 10, and joined ANBU Black Ops at 13. Itachi Uchiha, on the other hand, became a chunin at 10 and became a Captain of ANBU Black Ops at 13. Therefore, Kakashi is indeed the youngest Jounin.']


In [34]:
#Question 7
question = "Why did Kakashi's Sharingan fade after Obito died? In Chapter 691, after Obito died, Kakashi’s Sharingan faded. Why is it that Kakashi's Sharingan faded, while the Sharingan Danzo had stolen from Shisui and the one Shisui had given Itachi, were still okay even after its original owner had died?"

In [35]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [36]:
print(results["llm"]["replies"])

[" The Sharingan that Kakashi had was given to him by Obito, and it was Obito's Sharingan that faded after he died. The Sharingan that Danzo had stolen from Shisui and the one Shisui had given Itachi were not Obito's Sharingan, so they were not affected by his death. Additionally, the Sharingan that Itachi had was his own, and it did not fade after Obito's death. The Sharingan that Kakashi had was not his own, and it was not the same as the Sharingan that Danzo had stolen from Shisui or the one Shisui had given Itachi, so it faded after Obito's death."]


In [37]:
#Question 8
question = "Why can't the Inuzuka clan's dogs use human language?Why can't Inuzuka clan's dogs use human language when Kakashi's dogs can? Also, how does Kiba and Akamaru understand each other?"

In [38]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [39]:
print(results["llm"]["replies"])

[" The Inuzuka clan's dogs cannot use human language, while Kakashi's dogs can. This is because Kakashi's dogs are ninken, which are essentially dogs that have heightened senses and abilities, and as such, are able to work and be summoned alongside shinobi. They have shown the ability to use human language, walk on water, develop combat strategies, and track down targets through their enhanced smell. On the other hand, the Inuzuka clan's dogs are not ninken, but rather regular dogs that have been trained to work alongside shinobi. They do not have the ability to use human language. Kiba and Akamaru understand each other through their heightened senses and abilities, as well as their close bond as partners and best friends. Akamaru can also transform into an identical clone of Kiba, allowing them to communicate and coordinate their attacks more effectively."]


In [40]:
#Question 9
question = "How can Haku perform hand seals with only one hand?In the Land of the Waves arc, it can be seen that Haku performs handseals with only one hand. That must be something special, because a.) even Kakashi was surprised to see that, and b.) that's never seen again somewhere (except I forgot it). How is that possible? Is that a special technique? Was it ever explained in the manga/anime?"

In [41]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [42]:
print(results["llm"]["replies"])

[" Haku's ability to perform hand seals with only one hand is not explicitly explained in the manga or anime. It is possible that this is a unique skill that Haku possesses, but it is not mentioned as a specific technique or kekkei genkai. Kakashi's surprise at seeing Haku perform hand seals with one hand suggests that it is a rare and impressive ability, but it is not further explored or explained in the series."]


In [43]:
#Question 10
question = "Why can Naruto still use Rasenshuriken? When Naruto first used Rasenshuriken against Kakuzu the effect was massive. He destroyed two of Kakuzu's hearts, but there were a huge after effect to him. If I'm not mistaken, Tsunade advised Kakashi to ask Naruto never to use the technique again due to the huge risk. So why is Naruto still able to use the technique and have no side effects after that battle? I may have missed out on something."

In [44]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [45]:
print(results["llm"]["replies"])

[' Naruto is able to use Rasenshuriken without any side effects after the battle with Kakuzu because he learned senjutsu, which allowed him to infuse the Rasenshuriken with senjutsu chakra. This maintained the shape of the Rasenshuriken after formation, meaning that he could use it as a projectile and, as a result, that he no longer needed to worry about damaging himself. When thrown, the Rasenshuriken travels very quickly, enough to cross the crater of Chibaku Tensei in under a second.']


In [46]:
#Question 11
question = "What is the jutsu that ninjas use to appear/disappear instantly? Some users of the jutsu are Kakashi, Gai, Genma, Kabuto, etc. It can often be seen when Anbu members are dismissed."

In [47]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [48]:
print(results["llm"]["replies"])

[' The jutsu is called the Body Flicker Technique (瞬身の術, Shunshin no Jutsu). It is a high-speed movement technique used by ninjas to move at extremely high speeds, appearing and disappearing in an instant. It is often used by Anbu members when they are dismissed.']


In [50]:
#Question 12
question = "Why doesn't Kakashi use his copy ability that often? Kakashi is known as the copy cat ninja. The Ninjas from the other countries call Kakashi the Copy Cat Ninja because of his Sharingan and how he uses it for his advantage (Copying moves). But he does not use his power to copy very often (or at least not anymore). It is only shown around 3-4 times that he actually copies and uses it against somebody. Why does he not use this power more often?"

In [51]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [52]:
print(results["llm"]["replies"])

[" The document does not provide a clear answer as to why Kakashi does not use his copy ability more often. However, it does mention that Kakashi's use of Kamui, a technique he acquired by copying Obito's Sharingan, requires a great deal of chakra and can damage his vision with repeated use. It is possible that Kakashi refrains from using his copy ability excessively due to these limitations. Additionally, the document states that Kakashi has his own personal touches when copying techniques, which may also influence his decision to use certain techniques over others. Ultimately, the reason for Kakashi's limited use of his copy ability is not explicitly stated in the document."]


In [53]:
#Question 13
question = "Are people arware of who Naruto's parents are? Every villager in Konoha is aware that Naruto has the Kyuubi sealed inside of him. Is anyone in the village or elsewhere (other than Kakashi and Jiraiya, that I remember knew) aware that he is the son of Minato and Kushina? My main doubt being: do they know that he is the son of the Fourth Hokage?"

In [54]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [55]:
print(results["llm"]["replies"])

[' No, most people in Konoha are not aware that Naruto is the son of Minato and Kushina. The Third Hokage concealed Naruto\'s relation to Minato in order to keep him safe from his father\'s enemies and did his best to give Naruto a comfortable life. For years, Naruto did not know his parents\' identities, and neither did most of Konoha. Even when he was not aware of his relation to Minato, Naruto still greatly admired the Fourth Hokage, considering him a hero who gave his life for the village and becoming overjoyed whenever their similarities were remarked upon. However, in the alternate timeline of the movie "Road to Ninja: Naruto the Movie", Minato and Kushina are alive and have a son named Menma, who is the parallel version of Naruto. In this timeline, Kushina is alive alongside Minato, continuing to serve Konohagakure alongside her husband as ninja while also able to enjoy being parents, raising their son in a loving and stable lifestyle. Unbeknownst to the couple, Menma later disa

In [56]:
#Question 14
question = "Does every ninja become a jonin? In Naruto, a number of ninjas have become jonin, such as Neji and Kakashi. Does every ninja who becomes a chunin by definition become a jonin if they live long enough, or do some remain a chunin forever?"

In [57]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [58]:
print(results["llm"]["replies"])

[' Not every ninja who becomes a chunin becomes a jonin. Naruto, for example, remains a chunin for a long time.']


In [59]:
#Question 15
question = "Can a child inheritboth their parents' clans jutsu? Can a child in Naruto inherit both abilities from his parents' clans? For example, if Ino and Sasuke had a child is there a possibility for it to have both Uchiha's Sharingan and Yamanaka's mind transfer jutsu?"

In [60]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [61]:
print(results["llm"]["replies"])

[" In the Naruto universe, it is possible for a child to inherit abilities from both of their parents' clans. However, it is not guaranteed and depends on various factors such as genetics and the specific abilities in question. In the example you provided, if Ino (from the Yamanaka clan) and Sasuke (from the Uchiha clan) had a child, it is possible for that child to inherit both the Sharingan ability from the Uchiha clan and the mind transfer jutsu from the Yamanaka clan. However, this is not a certainty and would depend on the specific circumstances and genetics of the child. It is also worth noting that some abilities, such as kekkei genkai, may be more difficult or impossible for a child to inherit if they do not have the necessary prerequisites or if the abilities are incompatible."]
